In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
root_path = '../../../'
sys.path.append(root_path)

In [4]:
import argparse
import logging
import os
import numpy as np
np.random.seed(0)
import torch
from src.tc import utils
from src.ner.encoder import CharEncoder, ClassEncoder, WordEncoder
from src.tc.model.net import CNNTC
from src.tc.data_loader import DataLoader

In [5]:
data_dir = os.path.join(root_path, 'src/tc/data/sst_binary')
model_dir = os.path.join(root_path, 'src/tc/experiments/sst_binary_testjupyter')
restore_file = 'best'

In [6]:
# 1. set the device to train on
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [7]:
# 2. Load the parameters from json file
network_params = os.path.join(model_dir, 'params.json')
assert os.path.isfile(network_params), "No json configuration file found at {}".format(network_params)
params = utils.Params(network_params)
params.cuda = torch.cuda.is_available() # use GPU if available

In [8]:
# 3. Set the random seed for reproducible experiments
torch.manual_seed(230)
if params.cuda: torch.cuda.manual_seed(230)
np.random.seed(0)

In [9]:
# 4. Set the logger
utils.set_logger(os.path.join(model_dir, 'train.log'))

In [19]:
# 5. Create the input data pipeline
logging.info("Loading the datasets...")
# 5.1 specify features
from collections import OrderedDict
data_encoder = OrderedDict()
label_encoder = OrderedDict()
data_encoder[CharEncoder.FEATURE_NAME] = CharEncoder(os.path.join(data_dir, 'feats'))
data_encoder[WordEncoder.FEATURE_NAME] = WordEncoder(os.path.join(data_dir, 'feats'))
label_encoder[ClassEncoder.FEATURE_NAME] = ClassEncoder(os.path.join(data_dir, 'feats'))

# 5.2 load data
data_loader = DataLoader(params, data_dir, data_encoder, label_encoder)
data = data_loader.load_data(['test'], data_dir)
test_data = data['test']
test_data_iterator = data_loader.batch_iterator(test_data, params, shuffle=False)

# 5.3 specify the train and val dataset sizes
params.test_size = test_data['size']
logging.info("- done.")
logging.info('test size: {}'.format(params.test_size))

Loading the datasets...
- done.
test size: 1821


In [14]:
# 6. Modeling
# 6.1 Define the model
model = CNNTC(  params=params,
                char_vocab_length=data_encoder[CharEncoder.FEATURE_NAME].vocab_length,
                num_tags=label_encoder[ClassEncoder.FEATURE_NAME].num_tags,
                pretrained_word_vecs=torch.from_numpy(data_encoder[WordEncoder.FEATURE_NAME].vectors),
                dropout=params.dropout,
                decoder_type=params.decoder,
                bidirectional=params.rnn_bidirectional,
                freeze_embeddings=params.freeze_wordembeddings).to(device).float()
# 6.2 fetch metrics
from src.ner.evaluation import accuracy_score
metrics = {'accuracy': accuracy_score}

In [20]:
utils.load_checkpoint(os.path.join(model_dir, restore_file + '.pth'), model)

# Evaluate
from src.tc.evaluate import evaluate
num_steps = (params.test_size + 1) // params.batch_size
test_metrics = evaluate(model,
                        test_data_iterator,
                        metrics,
                        params,
                        num_steps,
                        label_encoder,
                        data_encoder)
save_path = os.path.join(model_dir, "metrics_test_{}.json".format(restore_file))
utils.save_dict_to_json(test_metrics, save_path)

- Eval metrics : accuracy: 0.812 ; loss: 0.413
